In [1]:
# To be run after Adult-Data-Prep
# Import Data handling/display libraries
import sys
import pandas as pd
import numpy  as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble      import RandomForestClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics       import accuracy_score
from sklearn.metrics       import balanced_accuracy_score
from sklearn.metrics       import auc, roc_auc_score, roc_curve, classification_report, confusion_matrix
from IPython.display       import Markdown, display
# Import IBM's AI Fairness tooolbox
from aif360.datasets       import BinaryLabelDataset
from aif360.metrics        import BinaryLabelDatasetMetric
from aif360.metrics        import ClassificationMetric
from aif360.metrics.utils  import compute_boolean_conditioning_vector
#from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
%matplotlib inline
# Warnings will be used to silence various model warnings for tidier output
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read the cleaned Taiwan-Credit-Card-Cleaned-Marriage dataset
Taiwan_df = pd.read_csv('./input/Taiwan-Credit-Card-Cleaned-Marriage.csv')

In [3]:
# Set privileged (1)/ unprivileged (0)/ favourable (1) / unfavourable values (0)
protected_attr      = 'MARRIAGE'
priv_grp            = 1  # Males 
unpriv_grp          = 0  # Females  
lab                 = 'DEFAULT'
fav_label           = 1 # Will not default next month
unfav_label         = 0 # Will default next month
privileged_groups   = [{protected_attr: priv_grp}]   # Males
unprivileged_groups = [{protected_attr: unpriv_grp}] # Females

In [4]:
# Create the traoning and test splits
X = Taiwan_df.drop(lab,axis=1)
y = Taiwan_df[lab]

In [5]:
# Create a Binary Label Dataset to use with AIF360 APIs
Taiwan_bld = BinaryLabelDataset(df=pd.concat((X, y), axis=1),
                                label_names=[lab], protected_attribute_names=[protected_attr],
                                favorable_label=fav_label, unfavorable_label=unfav_label)

In [6]:
# Create train and test datasets
Taiwan_train_bld, Taiwan_test_bld = Taiwan_bld.split([0.8], shuffle=True)

In [7]:
#Scale the binary labelled datasets
min_max_scaler = MinMaxScaler()
Taiwan_train_bld.features = min_max_scaler.fit_transform(Taiwan_train_bld.features)
Taiwan_test_bld.features  = min_max_scaler.fit_transform(Taiwan_test_bld.features)

In [8]:
# Convert the train and test datasets to dataframes
Taiwan_train_df, d = Taiwan_train_bld.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Taiwan_test_df,  d = Taiwan_test_bld.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)

In [9]:
# Determine the baseline model accuracy for Logistic Regression and Random Forest Classifiers
X_train = Taiwan_train_df.drop(lab,axis=1)
y_train = Taiwan_train_df[lab]
X_test  = Taiwan_test_df.drop(lab,axis=1)
y_test  = Taiwan_test_df[lab]
BiasedLogModel = LogisticRegression(random_state=101)
BiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
BiasedLogModel.fit(X_train, y_train) 
BiasedRfcModel.fit(X_train, y_train)
BiasedLogPredictions = BiasedLogModel.predict(X_test)
BiasedRfcPredictions = BiasedRfcModel.predict(X_test)
print(f"Biased Logistic regression validation accuracy: {BiasedLogModel.score(X_test, y_test)}")
print(f"Biased Random Forest       validation accuracy: {BiasedRfcModel.score(X_test, y_test)}")
print('')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_test, BiasedLogPredictions)}")
print(f"Biased Random forest balanced accuracy        : {balanced_accuracy_score(y_test, BiasedRfcPredictions)}")

Biased Logistic regression validation accuracy: 0.8153586497890295
Biased Random Forest       validation accuracy: 0.8114767932489452

Biased Logistic regression balanced accuracy  : 0.6002291903252249
Biased Random forest balanced accuracy        : 0.5866584361661615


In [10]:
print('Training Before - DEFAULT value counts:')
print(Taiwan_train_df.DEFAULT.value_counts())
print('Training Before - MARRIAGE value counts:')
print(Taiwan_train_df.MARRIAGE.value_counts())

Training Before - DEFAULT value counts:
1.0    18403
0.0     5295
Name: DEFAULT, dtype: int64
Training Before - MARRIAGE value counts:
0.0    12749
1.0    10949
Name: MARRIAGE, dtype: int64


In [11]:
print('Test Before - DEFAULT value counts:')
print(Taiwan_test_df.DEFAULT.value_counts())
print('Test Before - MARRIAGE value counts:')
print(Taiwan_test_df.MARRIAGE.value_counts())

Test Before - DEFAULT value counts:
1.0    4673
0.0    1252
Name: DEFAULT, dtype: int64
Test Before - MARRIAGE value counts:
0.0    3215
1.0    2710
Name: MARRIAGE, dtype: int64


In [12]:
# Create the binary label dataset metric class for the training dataset
metric_train_bld = BinaryLabelDatasetMetric(Taiwan_train_bld, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
metric_test_bld = BinaryLabelDatasetMetric(Taiwan_test_bld, 
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)

In [13]:
display(Markdown("#### Orig training dataset"))
print('Number of instances           :', metric_train_bld.num_instances())
print('Base Rate                     :', metric_train_bld.base_rate())
print('Consistency                   :', metric_train_bld.consistency())
print('Disparate Impact              :', metric_train_bld.disparate_impact())
print('Mean Difference               :', metric_train_bld.mean_difference())
print('Statistical Parity Difference :', metric_train_bld.statistical_parity_difference()) 
display(Markdown("#### Orig test dataset"))
print('Number of instances           :', metric_test_bld.num_instances())
print('Base Rate                     :', metric_test_bld.base_rate())
print('Consistency                   :', metric_test_bld.consistency())
print('Disparate Impact              :', metric_test_bld.disparate_impact())
print('Mean Difference               :', metric_test_bld.mean_difference())
print('Statistical Parity Difference :', metric_test_bld.statistical_parity_difference()) 

#### Orig training dataset

Number of instances           : 23698.0
Base Rate                     : 0.7765634230736771
Consistency                   : [0.78331505]
Disparate Impact              : 1.030574947054671
Mean Difference               : 0.023359159020214015
Statistical Parity Difference : 0.023359159020214015


#### Orig test dataset

Number of instances           : 5925.0
Base Rate                     : 0.7886919831223629
Consistency                   : [0.78872574]
Disparate Impact              : 1.0435622650053926
Mean Difference               : 0.03356384108164567
Statistical Parity Difference : 0.03356384108164567


In [14]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to FALSE!
sess = tf.Session()
biased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)

In [15]:
biased_model.fit(Taiwan_train_bld)

W0831 16:46:39.520472  2232 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:138: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0831 16:46:39.526547  2232 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:142: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0831 16:46:39.540892  2232 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:87: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0831 16:46:41.917606  2232 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https:

epoch 0; iter: 0; batch classifier loss: 0.677309
epoch 1; iter: 0; batch classifier loss: 0.452899
epoch 2; iter: 0; batch classifier loss: 0.498052
epoch 3; iter: 0; batch classifier loss: 0.512399
epoch 4; iter: 0; batch classifier loss: 0.429060
epoch 5; iter: 0; batch classifier loss: 0.451899
epoch 6; iter: 0; batch classifier loss: 0.357156
epoch 7; iter: 0; batch classifier loss: 0.508569
epoch 8; iter: 0; batch classifier loss: 0.518928
epoch 9; iter: 0; batch classifier loss: 0.475040
epoch 10; iter: 0; batch classifier loss: 0.435826
epoch 11; iter: 0; batch classifier loss: 0.500367
epoch 12; iter: 0; batch classifier loss: 0.514661
epoch 13; iter: 0; batch classifier loss: 0.426351
epoch 14; iter: 0; batch classifier loss: 0.491395
epoch 15; iter: 0; batch classifier loss: 0.447138
epoch 16; iter: 0; batch classifier loss: 0.436185
epoch 17; iter: 0; batch classifier loss: 0.310271
epoch 18; iter: 0; batch classifier loss: 0.356293
epoch 19; iter: 0; batch classifier loss:

In [16]:
# Apply the plain model to test data
Taiwan_biased_train = biased_model.predict(Taiwan_train_bld)
Taiwan_biased_test  = biased_model.predict(Taiwan_test_bld)
# And convert them to dataframes
Taiwan_train_df, d = Taiwan_biased_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Taiwan_test_df, d  = Taiwan_biased_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)

In [17]:
# Test whether it is possible to predict the Sensitive Variable from the biased dataset
X_se_train = Taiwan_train_df.drop(protected_attr,axis=1)
y_se_train = Taiwan_train_df[protected_attr]
X_se_test  = Taiwan_test_df.drop(protected_attr,axis=1)
y_se_test  = Taiwan_test_df[protected_attr]

Se_BiasedLogModel = LogisticRegression(random_state=101)
Se_BiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
Se_BiasedLogModel.fit(X_se_train, y_se_train) 
Se_BiasedRfcModel.fit(X_se_train, y_se_train) 
yseLog_pred =  Se_BiasedLogModel.predict(X_se_test)
yseRfc_pred =  Se_BiasedRfcModel.predict(X_se_test)
# Now test whether we can predict Gender from the test dataset
print('Model Accuracy for predicting the Sensitive Variable before bias transformation:')
print(f"Biased Logistic regression validation accuracy: {Se_BiasedLogModel.score(X_se_test, y_se_test)}")
print(f"Biased Random Forest       validation accuracy: {Se_BiasedRfcModel.score(X_se_test, y_se_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_se_test, yseLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_se_test, yseRfc_pred)}")

Model Accuracy for predicting the Sensitive Variable before bias transformation:
Biased Logistic regression validation accuracy: 0.7259071729957806
Biased Random Forest       validation accuracy: 0.7211814345991561

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.7278138683408607
Biased Random Forest       balanced accuracy  : 0.7277484462247421


In [18]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_biased_train = BinaryLabelDatasetMetric(Taiwan_biased_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

metric_biased_test = BinaryLabelDatasetMetric(Taiwan_biased_test, 
                            unprivileged_groups=unprivileged_groups,
                            privileged_groups=privileged_groups)
display(Markdown("#### Biased training dataset"))
print('Number of instances           :', metric_biased_train.num_instances())
print('Base Rate                     :', metric_biased_train.base_rate())
print('Consistency                   :', metric_biased_train.consistency())
print('Disparate Impact              :', metric_biased_train.disparate_impact())
print('Mean Difference               :', metric_biased_train.mean_difference())
print('Statistical Parity Difference :', metric_biased_train.statistical_parity_difference()) 
display(Markdown("#### Biased test dataset"))
print('Number of instances           :', metric_biased_test.num_instances())
print('Base Rate                     :', metric_biased_test.base_rate())
print('Consistency                   :', metric_biased_test.consistency())
print('Disparate Impact              :', metric_biased_test.disparate_impact())
print('Mean Difference               :', metric_biased_test.mean_difference())
print('Statistical Parity Difference :', metric_biased_test.statistical_parity_difference()) 

#### Plain model - without debiasing - dataset metrics

#### Biased training dataset

Number of instances           : 23698.0
Base Rate                     : 0.8863617182884631
Consistency                   : [0.97359271]
Disparate Impact              : 1.0163804376487724
Mean Difference               : 0.014392164597788804
Statistical Parity Difference : 0.014392164597788804


#### Biased test dataset

Number of instances           : 5925.0
Base Rate                     : 0.870210970464135
Consistency                   : [0.95986498]
Disparate Impact              : 1.0120122213084342
Mean Difference               : 0.010385473994708905
Statistical Parity Difference : 0.010385473994708905


In [19]:
# Create the Classification metric for train and test
classified_metric_biased_test = ClassificationMetric(Taiwan_train_bld, Taiwan_biased_train,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
classified_metric_biased_train = ClassificationMetric(Taiwan_test_bld, Taiwan_biased_test,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)

In [20]:
display(Markdown("#### Plain model - without debiasing - classification metrics"))

display(Markdown("#### Biased training dataset - CLassification Metrics"))
TPR_train = classified_metric_biased_train.true_positive_rate()
TNR_train = classified_metric_biased_train.true_negative_rate()
bal_acc_biased_train = 0.5*(TPR_train + TNR_train)
print('Classification accuracy         =', classified_metric_biased_train.accuracy())
print('Balanced classification accuracy=', bal_acc_biased_train)
print('Disparate impact                =', classified_metric_biased_train.disparate_impact())
print('Equal opportunity difference    =', classified_metric_biased_train.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_biased_train.average_odds_difference())
print('Theil_index                     =', classified_metric_biased_train.theil_index())

display(Markdown("#### Biased test dataset - CLassification Metrics"))
TPR_test = classified_metric_biased_test.true_positive_rate()
TNR_test = classified_metric_biased_test.true_negative_rate()
bal_acc_biased_test = 0.5*(TPR_test+TNR_test)
print('Classification accuracy         =', classified_metric_biased_test.accuracy())
print('Balanced classification accuracy=', bal_acc_biased_test)
print('Disparate impact                =', classified_metric_biased_test.disparate_impact())
print('Equal opportunity difference    =', classified_metric_biased_test.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_biased_test.average_odds_difference())
print('Theil_index                     =', classified_metric_biased_test.theil_index())

#### Plain model - without debiasing - classification metrics

#### Biased training dataset - CLassification Metrics

Classification accuracy         = 0.8222784810126582
Balanced classification accuracy= 0.6627964056995219
Disparate impact                = 1.0120122213084342
Equal opportunity difference    = 0.0005676723211572865
Average odds difference         = -0.0020384217886889044
Theil_index                     = 0.08778111604104136


#### Biased test dataset - CLassification Metrics

Classification accuracy         = 0.8199004135370074
Balanced classification accuracy= 0.6530052142317602
Disparate impact                = 1.0163804376487724
Equal opportunity difference    = 0.0029288702928870203
Average odds difference         = 0.012540098108827002
Theil_index                     = 0.0768833571517733


In [21]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [22]:
unbiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=True,
                          sess=sess)

In [23]:
unbiased_model.fit(Taiwan_train_bld)

epoch 0; iter: 0; batch classifier loss: 0.703472; batch adversarial loss: 0.807544
epoch 1; iter: 0; batch classifier loss: 0.527477; batch adversarial loss: 0.737845
epoch 2; iter: 0; batch classifier loss: 0.490016; batch adversarial loss: 0.710539
epoch 3; iter: 0; batch classifier loss: 0.473594; batch adversarial loss: 0.704102
epoch 4; iter: 0; batch classifier loss: 0.478514; batch adversarial loss: 0.684847
epoch 5; iter: 0; batch classifier loss: 0.417504; batch adversarial loss: 0.682009
epoch 6; iter: 0; batch classifier loss: 0.429960; batch adversarial loss: 0.681300
epoch 7; iter: 0; batch classifier loss: 0.460244; batch adversarial loss: 0.694946
epoch 8; iter: 0; batch classifier loss: 0.414330; batch adversarial loss: 0.693604
epoch 9; iter: 0; batch classifier loss: 0.438703; batch adversarial loss: 0.711385
epoch 10; iter: 0; batch classifier loss: 0.385647; batch adversarial loss: 0.690745
epoch 11; iter: 0; batch classifier loss: 0.489671; batch adversarial loss:

In [24]:
Taiwan_unbiased_train = unbiased_model.predict(Taiwan_train_bld)
Taiwan_unbiased_test  = unbiased_model.predict(Taiwan_test_bld)
# And convert these to dataframes..
Taiwan_train_df, d = Taiwan_unbiased_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Taiwan_test_df, d  = Taiwan_unbiased_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)

In [25]:
# Metrics for the dataset from plain model (after unbiasing)
display(Markdown("#### Plain model - after unbiasing - dataset metrics"))
metric_unbiased_train = BinaryLabelDatasetMetric(Taiwan_unbiased_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

metric_unbiased_test = BinaryLabelDatasetMetric(Taiwan_unbiased_test, 
                            unprivileged_groups=unprivileged_groups,
                            privileged_groups=privileged_groups)
display(Markdown("#### Biased training dataset"))
print('Number of instances           :', metric_unbiased_train.num_instances())
print('Base Rate                     :', metric_unbiased_train.base_rate())
print('Consistency                   :', metric_unbiased_train.consistency())
print('Disparate Impact              :', metric_unbiased_train.disparate_impact())
print('Mean Difference               :', metric_unbiased_train.mean_difference())
print('Statistical Parity Difference :', metric_unbiased_train.statistical_parity_difference()) 
display(Markdown("#### Biased test dataset"))
print('Number of instances           :', metric_unbiased_test.num_instances())
print('Base Rate                     :', metric_unbiased_test.base_rate())
print('Consistency                   :', metric_unbiased_test.consistency())
print('Disparate Impact              :', metric_unbiased_test.disparate_impact())
print('Mean Difference               :', metric_unbiased_test.mean_difference())
print('Statistical Parity Difference :', metric_unbiased_test.statistical_parity_difference()) 

#### Plain model - after unbiasing - dataset metrics

#### Biased training dataset

Number of instances           : 23698.0
Base Rate                     : 0.880327453793569
Consistency                   : [0.97370242]
Disparate Impact              : 1.0151071607454623
Mean Difference               : 0.013192032504097506
Statistical Parity Difference : 0.013192032504097506


#### Biased test dataset

Number of instances           : 5925.0
Base Rate                     : 0.860590717299578
Consistency                   : [0.95632068]
Disparate Impact              : 1.0241780424892548
Mean Difference               : 0.020537953435521894
Statistical Parity Difference : 0.020537953435521894


In [26]:
# Create the Classification metric for train and test
classified_metric_unbiased_train = ClassificationMetric(Taiwan_train_bld, Taiwan_unbiased_train,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
classified_metric_unbiased_test = ClassificationMetric(Taiwan_test_bld, Taiwan_unbiased_test,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)

In [27]:
display(Markdown("#### Plain model - after debiasing - classification metrics"))
display(Markdown("#### Unbiased training dataset - CLassification Metrics"))
TPR_train = classified_metric_unbiased_train.true_positive_rate()
TNR_train = classified_metric_unbiased_train.true_negative_rate()
bal_acc_unbiased_train = 0.5*(TPR_train + TNR_train)
print('Classification accuracy         =', classified_metric_unbiased_train.accuracy())
print('Balanced classification accuracy=', bal_acc_unbiased_train)
print('Disparate impact                =', classified_metric_unbiased_train.disparate_impact())
print('Equal opportunity difference    =', classified_metric_unbiased_train.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_unbiased_train.average_odds_difference())
print('Theil_index                     =', classified_metric_unbiased_train.theil_index())

display(Markdown("#### Unbiased test dataset - CLassification Metrics"))
TPR_test = classified_metric_unbiased_test.true_positive_rate()
TNR_test = classified_metric_unbiased_test.true_negative_rate()
bal_acc_unbiased_test = 0.5*(TPR_test+TNR_test)
print('Classification accuracy         =', classified_metric_unbiased_test.accuracy())
print('Balanced classification accuracy=', bal_acc_unbiased_test)
print('Disparate impact                =', classified_metric_unbiased_test.disparate_impact())
print('Equal opportunity difference    =', classified_metric_unbiased_test.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_unbiased_test.average_odds_difference())
print('Theil_index                     =', classified_metric_unbiased_test.theil_index())

#### Plain model - after debiasing - classification metrics

#### Unbiased training dataset - CLassification Metrics

Classification accuracy         = 0.8203645877289223
Balanced classification accuracy= 0.6584830387253136
Disparate impact                = 1.0151071607454623
Equal opportunity difference    = 0.002709703128113139
Average odds difference         = 0.009567080038121067
Theil_index                     = 0.0792443954176491


#### Unbiased test dataset - CLassification Metrics

Classification accuracy         = 0.8194092827004219
Balanced classification accuracy= 0.6668247132428902
Disparate impact                = 1.0241780424892548
Equal opportunity difference    = 0.010496009248760485
Average odds difference         = 0.007820351891261235
Theil_index                     = 0.09384162020093618


In [28]:
# Determine the after-transformation model accuracy for Logistic Regression and Random Forest Classifiers
X_train = Taiwan_train_df.drop(lab,axis=1)
y_train = Taiwan_train_df[lab]
X_test  = Taiwan_test_df.drop(lab,axis=1)
y_test  = Taiwan_test_df[lab]

UnbiasedLogModel = LogisticRegression(random_state=101)
UnbiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
UnbiasedLogModel.fit(X_train, y_train) 
UnbiasedRfcModel.fit(X_train, y_train) 
UnbiasedLog_pred = UnbiasedLogModel.predict(X_test)
UnbiasedRfc_pred = UnbiasedRfcModel.predict(X_test)

# Now get Logistic Regression and Random Forest Clasisfication performancce of unbiased data
print('Model Accuracy for Log Reg and RFC after bias transformation:')
print(f"Biased Logistic regression validation accuracy: {UnbiasedLogModel.score(X_test, y_test)}")
print(f"Biased Random Forest       validation accuracy: {UnbiasedRfcModel.score(X_test, y_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_test, UnbiasedLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_test, UnbiasedRfc_pred)}")

Model Accuracy for Log Reg and RFC after bias transformation:
Biased Logistic regression validation accuracy: 0.9738396624472574
Biased Random Forest       validation accuracy: 0.9409282700421941

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.9158124343803822
Biased Random Forest       balanced accuracy  : 0.7891501300877017


In [29]:
# Finally test whether it is possible to predict the Sensitive Variable from the transformed dataset
X_se_train = Taiwan_train_df.drop(protected_attr,axis=1)
y_se_train = Taiwan_train_df[protected_attr]
X_se_test  = Taiwan_test_df.drop(protected_attr,axis=1)
y_se_test  = Taiwan_test_df[protected_attr]

Se_unbiasedLogModel = LogisticRegression(random_state=101)
Se_unbiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
Se_unbiasedLogModel.fit(X_se_train, y_se_train) 
Se_unbiasedRfcModel.fit(X_se_train, y_se_train) 
yseLog_pred =  Se_unbiasedLogModel.predict(X_se_test)
yseRfc_pred =  Se_unbiasedRfcModel.predict(X_se_test)
# Now test whether we can predict Gender from the test dataset
print('Model Accuracy for predicting the Sensitive Variable before bias transformation:')
print(f"Biased Logistic regression validation accuracy: {Se_unbiasedLogModel.score(X_se_test, y_se_test)}")
print(f"Biased Random Forest       validation accuracy: {Se_unbiasedRfcModel.score(X_se_test, y_se_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_se_test, yseLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_se_test, yseRfc_pred)}")

Model Accuracy for predicting the Sensitive Variable before bias transformation:
Biased Logistic regression validation accuracy: 0.7270886075949367
Biased Random Forest       validation accuracy: 0.7220253164556962

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.729134362105674
Biased Random Forest       balanced accuracy  : 0.728526051201414


In [30]:
print('Predicted Training After - DEFAULT value counts:')
print(Taiwan_train_df.DEFAULT.value_counts())
print('Predicted Training After - MARRIAGE value counts:')
print(Taiwan_train_df.MARRIAGE.value_counts())

Predicted Training After - DEFAULT value counts:
1.0    20862
0.0     2836
Name: DEFAULT, dtype: int64
Predicted Training After - MARRIAGE value counts:
0.0    12749
1.0    10949
Name: MARRIAGE, dtype: int64


In [31]:
print('Predicted Test After - DEFAULT value counts:')
print(Taiwan_test_df.DEFAULT.value_counts())
print('Predicted Test After - MARRIAGE value counts:')
print(Taiwan_test_df.MARRIAGE.value_counts())

Predicted Test After - DEFAULT value counts:
1.0    5099
0.0     826
Name: DEFAULT, dtype: int64
Predicted Test After - MARRIAGE value counts:
0.0    3215
1.0    2710
Name: MARRIAGE, dtype: int64
